# Advent of Code 2021

Here, I will solve problems from Advent of Code 2021 and compare my solution with Norvig's. After a quick glance at his solutions, I make improvements on mine. 

Import libraries and utilities.

In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from typing      import *
from statistics  import mean, median
from math        import ceil, inf
from functools   import lru_cache
import matplotlib.pyplot as plt
import re

def answer(puzzle_number, got, expected) -> bool:
    """Verify the answer we got was the expected answer."""
    assert got == expected, f'For {puzzle_number}, expected {expected} but got {got}.'
    return True

# Now able to parse given strings
def parse(inp, parser=str, sep='\n', print_lines=7) -> tuple:
    """Split the day's input file into entries separated by `sep`, 
    and apply `parser` to each. If a string is given, parse it instad."""
    if isinstance(inp, int):
        title = f'input{inp}.txt' # Suited to my needs
        text  = open(title).read()
    elif isinstance(inp, str):
        title = 'sample'
        text = inp.strip()
    entries = mapt(parser, text.rstrip().split(sep))
    if print_lines:
        all_lines = text.splitlines()
        lines = all_lines[:print_lines]
        head = f'{title} ➜ {len(text)} chars, {len(all_lines)} lines; first {len(lines)} lines:'
        dash = "-" * 100
        print(f'{dash}\n{head}\n{dash}')
        for line in lines:
            print(trunc(line))
        print(f'{dash}\nparse({inp}) ➜ {len(entries)} entries:\n'
              f'{dash}\n{trunc(str(entries))}\n{dash}')
    return entries

def trunc(s: str, left=70, right=25, dots=' ... ') -> str: 
    """All of string s if it fits; else left and right ends of s with dots in the middle."""
#     dots = ' ... ' No need
    return s if len(s) <= left + right + len(dots) else s[:left] + dots + s[-right:]

Char = str # Intended as the type of a one-character string
Atom = Union[float, int, str]

def ints(text: str) -> Tuple[int]:
    """A tuple of all the integers in text, ignoring non-number characters."""
    return mapt(int, re.findall(r'-?[0-9]+', text))

def digits(text: str) -> Tuple[int]:
    """A tuple of all the digits in text (as ints 0–9), ignoring non-digit characters."""
    return mapt(int, re.findall(r'[0-9]', text))

def words(text: str) -> List[str]:
    """A list of all the alphabetic words in text, ignoring non-letters."""
    return re.findall(r'[a-zA-Z]+', text)

def atoms(text: str) -> Tuple[Atom]:
    """A tuple of all the atoms (numbers or symbol names) in text."""
    return mapt(atom, re.findall(r'[a-zA-Z_0-9.+-]+', text))

def atom(text: str) -> Atom:
    """Parse text into a single float or int or str."""
    try:
        x = float(text)
        return round(x) if round(x) == x else x
    except ValueError:
        return text
    
def mapt(fn, *args) -> tuple:
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

def quantify(iterable, pred=bool) -> int:
    """Count the number of items in iterable for which pred is true."""
    return sum(1 for item in iterable if pred(item))

class multimap(defaultdict):
    """A mapping of {key: [val1, val2, ...]}."""
    def __init__(self, pairs: Iterable[tuple], symmetric=False):
        """Given (key, val) pairs, return {key: [val, ...], ...}.
        If `symmetric` is True, treat (key, val) as (key, val) plus (val, key)."""
        self.default_factory = list
        for (key, val) in pairs:
            self[key].append(val)
            if symmetric:
                self[val].append(key)

def prod(numbers) -> float: # Will be math.prod in Python 3.8
    """The product formed by multiplying `numbers` together."""
    result = 1
    for x in numbers:
        result *= x
    return result

def total(counter: Counter) -> int: 
    """The sum of all the counts in a Counter."""
    return sum(counter.values())

def sign(x) -> int: return (0 if x == 0 else +1 if x > 0 else -1)

def transpose(matrix) -> list: return list(zip(*matrix))

def nothing(*args) -> None: return None

cat     = ''.join
flatten = chain.from_iterable
cache   = lru_cache(None)

Point = Tuple[int, int] # (x, y) points on a grid

neighbors4 = ((0, 1), (1, 0), (0, -1), (-1, 0))               
neighbors8 = ((1, 1), (1, -1), (-1, 1), (-1, -1)) + neighbors4

class Grid(dict):
    """A 2D grid, implemented as a mapping of {(x, y): cell_contents}."""
    def __init__(self, mapping=(), rows=(), neighbors=neighbors4):
        """Initialize with, e.g., either `mapping={(0, 0): 1, (1, 0): 2, ...}`,
        or `rows=[(1, 2, 3), (4, 5, 6)].
        `neighbors` is a collection of (dx, dy) deltas to neighboring points.`"""
        self.update(mapping if mapping else
                    {(x, y): val 
                     for y, row in enumerate(rows) 
                     for x, val in enumerate(row)})
        self.width  = max(x for x, y in self) + 1
        self.height = max(y for x, y in self) + 1
        self.deltas = neighbors
        
    def copy(self) -> Grid: return Grid(self, neighbors=self.deltas)
    
    def neighbors(self, point) -> List[Point]:
        """Points on the grid that neighbor `point`."""
        x, y = point
        return [(x+dx, y+dy) for (dx, dy) in self.deltas 
                if (x+dx, y+dy) in self]
    
    def to_rows(self) -> List[List[object]]:
        """The contents of the grid in a rectangular list of lists."""
        return [[self[x, y] for x in range(self.width)]
                for y in range(self.height)]

## Day 1

Problem 1.1: How many times the value increased from the previous one?

In [3]:
inp1 = parse(1, parser=int)

----------------------------------------------------------------------------------------------------
input1.txt ➜ 9797 chars, 2000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
178
205
212
210
215
220
223
----------------------------------------------------------------------------------------------------
parse(1) ➜ 2000 entries:
----------------------------------------------------------------------------------------------------
(178, 205, 212, 210, 215, 220, 223, 224, 230, 232, 235, 225, 226, 227, ... , 8741, 8750, 8753, 8755)
----------------------------------------------------------------------------------------------------


In [4]:
# A sample provided by Advent of Code
sample1 = [199, 200, 208, 210, 200, 207, 240, 269, 260, 263]

In [5]:
# My naive attempt to solve
def deeper(inputs):
    return sum(1 for x, y in zip(inputs[:-1], inputs[1:]) if y > x)

In [6]:
# Way better version with quantify adopted from Norvig
def deeper_quant(inputs):
    return quantify(inputs[i + 1] > inputs[i] for i in range(len(inputs) - 1))

In [7]:
deeper(sample1)

7

In [8]:
answer('sample1', deeper_quant(sample1), 7)

True

In [9]:
deeper(inp1)

1676

In [11]:
answer(1.1, deeper_quant(inp1), 1676)

True

Problem 1.2: Three-measurement sliding window. Instead of comparing one item to another, we now compare a tripplet to next tripplet.

In [12]:
# My naive version
def deeper2(inputs):
    three_measurements = [x+y+z for x, y, z in 
                          zip(inputs[:-2], inputs[1:-1], inputs[2:])]
    return sum(1 for x, y in zip(three_measurements[:-1], three_measurements[1:]) if y > x)

In [13]:
# Better version with quantify
def deeper_quant2(inputs):
    return quantify(sum(inputs[i:i+3]) < sum(inputs[i+1:i+4])
                   for i in range(len(inputs) - 3))

In [14]:
deeper2(sample1)

5

In [15]:
answer('sample1', deeper_quant2(sample1), 5)

True

In [16]:
deeper2(inp1[:10])

7

In [17]:
deeper2(inp1)

1706

In [88]:
answer('sample1', deeper_quant2(inp1), 1611)

True

## Day 2

Problem 2.1: What is the product of distance travelled? 

In [18]:
inp2 = parse(2, parser=lambda x: x.split() )

----------------------------------------------------------------------------------------------------
input2.txt ➜ 7733 chars, 1000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
forward 6
forward 9
down 9
down 7
forward 8
down 4
forward 7
----------------------------------------------------------------------------------------------------
parse(2) ➜ 1000 entries:
----------------------------------------------------------------------------------------------------
(['forward', '6'], ['forward', '9'], ['down', '9'], ['down', '7'], ['f ... , '3'], ['forward', '5'])
----------------------------------------------------------------------------------------------------


In [19]:
point = [0, 0]
point

[0, 0]

In [20]:
def dive(point, direction):
    """Follow direction from the point."""
    heading, distance = direction
    distance = int(distance)
    if heading == 'forward':
        point[0] += distance
    elif heading == 'down':
        point[1] += distance
    elif heading == 'up':
        point[1] -= distance
    else:
        raise ValueError('Invalid direction')

In [21]:
sample_inp2 = '''
forward 5
down 5
forward 8
up 3
down 8
forward 2
'''
samples2 = parse(sample_inp2, parser=lambda x: x.split())

----------------------------------------------------------------------------------------------------
sample ➜ 48 chars, 6 lines; first 6 lines:
----------------------------------------------------------------------------------------------------
forward 5
down 5
forward 8
up 3
down 8
forward 2
----------------------------------------------------------------------------------------------------
parse(
forward 5
down 5
forward 8
up 3
down 8
forward 2
) ➜ 6 entries:
----------------------------------------------------------------------------------------------------
(['forward', '5'], ['down', '5'], ['forward', '8'], ['up', '3'], ['down', '8'], ['forward', '2'])
----------------------------------------------------------------------------------------------------


In [22]:
def solve2_1(directions):
    """Solve problem 2.1"""
    point = [0, 0]
    for direction in directions:
        dive(point, direction)
    return prod(point)

In [23]:
answer('sample 2.1', solve2_1(samples2), 150)

True

In [24]:
solve2_1(inp2)

2027977

Problem 2.2: Now, down increases aim by X, up decreases aim by X, and forward moves horizontal distance and increases depth by aim multiplied by X.

In [25]:
def dive2(point, direction):
    """Follow direction from the point with aim."""
    heading, distance = direction
    distance = int(distance)
    if heading == 'forward':
        point[0] += distance
        point[1] += point[2] * distance
    elif heading == 'down':
        point[2] += distance
    elif heading == 'up':
        point[2] -= distance
    else:
        raise ValueError('Invalid direction')

In [26]:
def solve2_2(directions):
    """Solve problem 2.2"""
    point = [0, 0, 0] # x, y, aim
    for direction in directions:
        dive2(point, direction)
    return point[0] * point[1]

In [27]:
answer('sample 2.2', solve2_2(samples2), 900)

True

In [28]:
solve2_2(inp2)

1903644897

I used a list to store x, y, and aim, but I did not have to. Because it is such a simple problem, Norvig did not bother making a list. When parsing the input, I splitted the inputs by using lambda function, but atoms function can do exactly what I wanted to achieve by parsing integer as int.

In [97]:
inp2 = parse(2, atoms)

----------------------------------------------------------------------------------------------------
input2.txt ➜ 7793 chars, 1000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
forward 6
down 2
forward 2
down 8
forward 3
down 6
down 8
----------------------------------------------------------------------------------------------------
parse(2) ➜ 1000 entries:
----------------------------------------------------------------------------------------------------
(('forward', 6), ('down', 2), ('forward', 2), ('down', 8), ('forward', ... own', 9), ('forward', 8))
----------------------------------------------------------------------------------------------------


In [104]:
sample_inp2 = '''
forward 5
down 5
forward 8
up 3
down 8
forward 2
'''
samples2 = parse(sample_inp2, atoms)

----------------------------------------------------------------------------------------------------
sample ➜ 48 chars, 6 lines; first 6 lines:
----------------------------------------------------------------------------------------------------
forward 5
down 5
forward 8
up 3
down 8
forward 2
----------------------------------------------------------------------------------------------------
parse(
forward 5
down 5
forward 8
up 3
down 8
forward 2
) ➜ 6 entries:
----------------------------------------------------------------------------------------------------
(('forward', 5), ('down', 5), ('forward', 8), ('up', 3), ('down', 8), ('forward', 2))
----------------------------------------------------------------------------------------------------


In [110]:
def improved_solve2_1(directions):
    x = y = 0
    for heading, n in directions:
        if heading == 'forward': x += n
        elif heading == 'down': y += n
        elif heading == 'up': y -= n
    return x * y

In [111]:
answer('improved sample 2.1', improved_solve2_1(samples2), 150)

True

In [112]:
answer('improved 2.1', improved_solve2_1(inp2), 2322630)

True

In [113]:
def improved_solve_2_2(directions):
    x = y = aim = 0
    for heading, n in directions:
        if heading == 'forward': 
            x += n
            y += n * aim
        elif heading == 'down': 
            aim += n
        elif heading == 'up': 
            aim -= n
    return x * y

In [114]:
answer('sample 2.2', solve2_2(samples2), 900)

True

In [115]:
answer('sample 2.2', solve2_2(inp2), 2105273490)

True

## Day 3

Problem 3.1: What is a value of the most and least common digits in binary, convert them to decimal, then multiplied together?

In [29]:
inp3 = parse(3, lambda x: x)

----------------------------------------------------------------------------------------------------
input3.txt ➜ 12999 chars, 1000 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
101001100010
010100001011
010010010101
110100011010
001100100001
111111110110
000000101100
----------------------------------------------------------------------------------------------------
parse(3) ➜ 1000 entries:
----------------------------------------------------------------------------------------------------
('101001100010', '010100001011', '010010010101', '110100011010', '0011 ... 1101111', '010100000111')
----------------------------------------------------------------------------------------------------


In [30]:
nums = list(zip('1001', '0111', '0000'))
nums

[('1', '0', '0'), ('0', '1', '0'), ('0', '1', '0'), ('1', '1', '0')]

In [31]:
c = Counter(nums[0])
c.most_common()

[('0', 2), ('1', 1)]

In [32]:
c.most_common()[0][0]

'0'

In [33]:
list(c.values())

[1, 2]

In [34]:
int('0b10', 2)

2

In [35]:
sample_inp3 = '''
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
'''
samples3 = parse(sample_inp3)

----------------------------------------------------------------------------------------------------
sample ➜ 71 chars, 12 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
00100
11110
10110
10111
10101
01111
00111
----------------------------------------------------------------------------------------------------
parse(
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
) ➜ 12 entries:
----------------------------------------------------------------------------------------------------
('00100', '11110', '10110', '10111', '10101', '01111', '00111', '11100 ... 11001', '00010', '01010')
----------------------------------------------------------------------------------------------------


In [36]:
def get_commons(iterable, rank=0):
    """Grab the most common item from an iterable and convert it to decimal number."""
    return int(cat(Counter(inp).most_common()[rank][0] for inp in zip(*iterable)), 2)

In [37]:
def solve3_1(iterable):
    return get_commons(iterable) * get_commons(iterable, 1)

In [38]:
answer('sample3.1', get_commons(samples3), 22)

True

In [39]:
answer('sample3.1', get_commons(samples3, 1), 9)

True

In [40]:
answer('sample3.1', solve3_1(samples3), 198)

True

In [41]:
solve3_1(inp3)

741950

Problem 3.2: We need to find O2 generator rating and CO2 scrubber rating. Here is how we find O2 generator rating. For first digit, find a most frequent digit, and filter numbers with that digit. Keep doing it until the last number is left or if there is a tie. If tie, keep the number with 1. To find CO2 scrubber rating, grab second frequent digits, and if there is a tie, grab a digit with 0. Then multiply O2 generator rating with CO2 scrubber rating.

In [42]:
l = list(range(10))
for n in l:
    if len(l) == 7:
        break
    if n % 2 == 0:
        l.remove(n)
    print(f'{n} checked')
l

0 checked
2 checked
4 checked


[1, 3, 5, 6, 7, 8, 9]

In [43]:
def get_commons2(iterable, rank=0):
    """For each digit, count occurrences and find a comparing number to compare.
    Compare the number with result and filter out ones that don't satisfy the condition."""
    result = list(iterable)
    for i in range(len(iterable[0])):
        if len(result) == 1:
            return result[0]
        compare = find_comparing_num(result, rank, i)
        result = list(filter(lambda x: x[i] == compare, result))
    return result[0]

def find_comparing_num(result, rank, i):
    """Find a number to compare with result by checking 'i'th digit of result."""
    most_common = Counter(res[i] for res in result).most_common()
    compare = most_common[rank][0] 
    if most_common[0][1] == most_common[1][1]:
        if rank == 0:
            compare = '1'
        else:
            compare = '0'
    return compare

In [44]:
answer(3.2, get_commons2(samples3), '10111')

True

In [45]:
answer(3.2, get_commons2(samples3, 1), '01010')

True

In [46]:
def solve3_2(inputs):
    return int(get_commons2(inputs), 2) * int(get_commons2(inputs, 1), 2)

In [47]:
answer('sample3.2', solve3_2(samples3), 230)

True

In [49]:
solve3_2(inp3)

903810

I used Counter to count occurrences, but Norvig used str.count instead. Also, I used iterative approaches, but his version used recursive one. I love recursion, and I wish python transforms tail recursion into iterative version. I wrote my own recursive version just for fun.

In [50]:
def get_commons2_rec(binaries, rank=0, i=0):
    """Solve problem 3.2 recursively."""
    if len(binaries) == 1:
        return binaries[0]
    compare = find_comparing_num(binaries, rank, i)
    return get_commons2_rec(list(filter(lambda x: x[i] == compare, binaries)), rank, i + 1)

In [51]:
def solve3_2_rec(inputs):
    return int(get_commons2_rec(inputs), 2) * int(get_commons2_rec(inputs, 1), 2)

In [52]:
answer(3.2, get_commons2_rec(samples3), '10111')

True

In [53]:
answer(3.2, get_commons2_rec(samples3, 1), '01010')

True

In [54]:
answer('sample3.2', solve3_2_rec(samples3), 230)

True

In [56]:
answer(3.2, solve3_2_rec(inp3), 903810)

True

## Day 4

Play bingo.

So, we keep track of boards with marked indices and check whether it is bingo. We have 100 boards, and we will just use one-dimension to solve this puzzle. A board has a bingo if a row or a column is marked.

Another strategy can be marking each number by setting it with -1. Then, we add each row and column and check for -5. If we do, return sum of list + 5. Here, we have to make sure index does not equal number of -1s. Sometimes the board does not include the draw.

In [84]:
inp4 = parse(4, parser=lambda x: list(atoms(x)), sep='\n\n')

----------------------------------------------------------------------------------------------------
input4.txt ➜ 7889 chars, 601 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
93,18,74,26,98,52,94,23,15,2,34,75,13,31,39,76,96,16,84,12,38,27,8,85, ... 37,73,70,68,97,61,95,53,1

97 18 90 62 17
98 88 49 41 74
66  9 83 69 91
33 57  3 71 43
11 50  7 10 28
----------------------------------------------------------------------------------------------------
parse(4) ➜ 101 entries:
----------------------------------------------------------------------------------------------------
([93, 18, 74, 26, 98, 52, 94, 23, 15, 2, 34, 75, 13, 31, 39, 76, 96, 1 ... , 9, 22, 96, 21, 12, 65])
----------------------------------------------------------------------------------------------------


In [85]:
# With 13, second one gets a bingo, and 3, 15, 22, 18, 19, 8, 25, 20, 12, 6
sample_inp4 = """
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
"""
samples4 = parse(sample_inp4, parser=lambda x: list(atoms(x)), sep='\n\n')

----------------------------------------------------------------------------------------------------
sample ➜ 298 chars, 19 lines; first 7 lines:
----------------------------------------------------------------------------------------------------
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6
----------------------------------------------------------------------------------------------------
parse(
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
) ➜ 4 entries:
----------------------------------------------------------------------------------------------------
([7, 4, 9, 5, 11, 17, 23, 2, 0, 14, 21, 24, 10, 16, 13, 6, 15, 25, 12, ... 3, 6, 5,

In [4]:
draws = inp4[0]

In [5]:
inp4[1]

[97,
 18,
 90,
 62,
 17,
 98,
 88,
 49,
 41,
 74,
 66,
 9,
 83,
 69,
 91,
 33,
 57,
 3,
 71,
 43,
 11,
 50,
 7,
 10,
 28]

In [6]:
88 in inp4[1]

True

In [4]:
# indicies that makes bingo
bingo_indices = [tuple(x for x in range(y, y + 5)) for y in range(0, 25, 5)]
bingo_indices += transpose(bingo_indices)
bingo_indices

[(0, 1, 2, 3, 4),
 (5, 6, 7, 8, 9),
 (10, 11, 12, 13, 14),
 (15, 16, 17, 18, 19),
 (20, 21, 22, 23, 24),
 (0, 5, 10, 15, 20),
 (1, 6, 11, 16, 21),
 (2, 7, 12, 17, 22),
 (3, 8, 13, 18, 23),
 (4, 9, 14, 19, 24)]

In [56]:
[1,2,3].count(2)

1

In [5]:
from IPython.core.debugger import set_trace

In [6]:
def solve4_1(inp):
    """Setup a bingo game."""
    draws = inp[0]
    boards = inp[1:]
    for i in range(len(inp[1]) - 1):
        draw = draws[i]
        for board in boards:
            if draw in board:
                board[board.index(draw)] = -1
            if is_bingo(board):
                print('bingo!')
#                 return (sum(board) + board.count(-1)) * draw

In [7]:
def is_bingo(board):
    """Check for -5 on sum of each row/column"""
    return any([sum(board[i] for i in bingo_index) == -5 for bingo_index in bingo_indices])

In [140]:
answer('sample4_1', solve4_1(samples4), 4512)

bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!


AssertionError: For sample4_1, expected 4512 but got None.

In [141]:
solve4_1(inp4)

bingo!
bingo!


Problem 4.2: Choose the board that will win last.

In [57]:
def solve4_2(inp):
    """Setup a bingo game."""
    draws = inp[0]
    boards = list(inp[1:])
    last_win = 0
    for i in range(len(inp[0]) - 1):
        draw = draws[i]
        for j in range(len(boards)):
            if draw in boards[j]:
                boards[j][boards[j].index(draw)] = -1
            if is_bingo(boards[j]):
                last_win = (sum(boards[j]) + boards[j].count(-1)) * draw
                boards[j] = reset(boards[j])
    return last_win

In [58]:
def reset(board):
    """Reset the board to 0s"""
    return [-5 for _ in range(25)]

In [59]:
answer('sample 4.2', solve4_2(samples4), 1924)

1924

In [60]:
answer(4.2, solve4_2(inp4), 31755)

31755

We managed to come to an answer with the strategy. However, I would like to improve this solution. Let's get some hints from Norvig's solutions. He has a much simpler solution. When I come up with strategies, I am not sure how complicated the code may turn out, so I do not know which strategy to implement. 

After studying his code, I will come back later to implement his strategy by looking at pseudocode I wrote. 

In [3]:
# Strategy
# For each draw, check for bingo by looking at possible rows and columns.
# It is bingo if either row or column are all marked.
# If it is bingo, return a sum of unmakred numbers in the board multiplied by the draw.

## Day 5